In [21]:
import pandas as pd
import os 
import glob
from pathlib import Path
import sqlalchemy
import psycopg2

data_path = "../data/MLB_Data_2025"

In [4]:
def load_all_csvs(file_name):
    all_dfs = []
    
    path_list = Path(data_path).rglob(file_name)
    
    for path in path_list:
        # path.parent.parent is the date-stamped folder
        folder_date = path.parent.parent.name 
        
        df = pd.read_csv(path)
        df['source_folder_date'] = folder_date
        all_dfs.append(df)
        
    if not all_dfs:
        print(f"Warning: No files named {file_name} found in {data_path}")
        return pd.DataFrame()
        
    return pd.concat(all_dfs, ignore_index=True)

df_games_raw = load_all_csvs('games.csv')
df_linescores_raw = load_all_csvs('linescores.csv')
df_runners_raw = load_all_csvs('runners.csv')

print("Sample Folder Dates captured:", df_games_raw['source_folder_date'].unique()[:5])

/var/folders/50/1xdm0_hs2r1b2s4b88l6x6bm0000gn/T/ipykernel_37025/4246475191.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_dfs, ignore_index=True)
/var/folders/50/1xdm0_hs2r1b2s4b88l6x6bm0000gn/T/ipykernel_37025/4246475191.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_dfs, ignore_index=True)


Sample Folder Dates captured: ['2025-07-22' '2025-04-05' '2025-03-12' '2025-03-15' '2025-04-02']


/var/folders/50/1xdm0_hs2r1b2s4b88l6x6bm0000gn/T/ipykernel_37025/4246475191.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_dfs, ignore_index=True)


In [ ]:
# Removing Duplicate Games

# Check how many games are duplicated across dates
dup_counts = df_games_raw['gamePk'].value_counts()
print(f"Games appearing more than once: {(dup_counts > 1).sum()}")

# Remove duplicates
df_games = df_games_raw.sort_values(['gamePk', 'source_folder_date'], ascending=[True, False])
df_games = df_games.drop_duplicates(subset=['gamePk'], keep='first')

# Validation: should be 0
print(f"Duplicates remaining: {df_games['gamePk'].duplicated().sum()}")

Games appearing more than once: 34
Duplicates remaining: 0


In [17]:
df_games[['hometeamid', 'awayteamid']].value_counts()

hometeamid  awayteamid
147         111           11
141         147           11
120         121           11
146         120           10
147         141           10
                          ..
133         119            1
            115            1
121         111            1
119         140            1
562         111            1
Name: count, Length: 752, dtype: int64

In [20]:
df_games[['venueid', 'venuename', 'scheduledInnings']].value_counts()

venueid  venuename                               scheduledInnings
2523     George M. Steinbrenner Field            9                   97
22       Dodger Stadium                          9                   91
14       Rogers Centre                           9                   91
680      T-Mobile Park                           9                   87
3313     Yankee Stadium                          9                   86
32       American Family Field                   9                   86
17       Wrigley Field                           9                   84
5        Progressive Field                       9                   84
2394     Comerica Park                           9                   83
2392     Daikin Park                             9                   83
1        Angel Stadium                           9                   83
2681     Citizens Bank Park                      9                   83
5325     Globe Life Field                        9                   8

In [14]:
df_games[['gameType', 'sportId', 'codedGameState', 'detailedState']].value_counts()

gameType  sportId  codedGameState  detailedState                     
R         1        F               Final                                 2425
S         1        F               Final                                  422
                                   Final: Tied                             33
D         1        F               Final                                   18
S         1        C               Cancelled                               13
E         1        F               Final                                   12
F         1        F               Final                                   11
L         1        F               Final                                   11
W         1        F               Final                                    7
R         1        F               Completed Early: Rain                    4
S         1        F               Completed Early                          2
A         1        F               Final: Tie, decision by tiebreaker   

In [15]:
df_games[['awayteamscore', 'hometeamscore']].describe()

,awayteamscore,hometeamscore
count,2948.000000,2948.000000
mean,4.473541,4.625509
std,3.363356,3.181031
min,0.000000,0.000000
25%,2.000000,2.000000
50%,4.000000,4.000000
75%,6.000000,6.000000
max,24.000000,22.000000


In [ ]:
# Only null values are the scores of the cancelled games
df_games.isnull().sum()

gamePk                 0
gameDate               0
officialDate           0
sportId                0
gameType               0
codedGameState         0
detailedState          0
awayteamid             0
awayteamname           0
awayteamscore         13
hometeamid             0
hometeamname           0
hometeamscore         13
venueid                0
venuename              0
scheduledInnings       0
source_folder_date     0
dtype: int64

In [122]:
from sqlalchemy import create_engine
# Load the games data into the game table

DB_URI = "postgresql://von@localhost:5432/mlb_data_2025"

engine = create_engine(DB_URI)

def load_games_to_db(df):
    # Ensure column names are all lowercase to match our SQL definition
    df.columns = [c.lower() for c in df.columns]
    
    try:
        df.to_sql('game', engine, if_exists='append', index=False)
        print("Successfully loaded games into the database.")
    except Exception as e:
        print(f"Error loading to database: {e}")

def load_linescores_to_db(df):
    # Ensure column names are all lowercase to match our SQL definition
    df.columns = [c.lower() for c in df.columns]
    
    try:
        df.to_sql('linescore', engine, if_exists='append', index=False)
        print("Successfully loaded linescores into the database.")
    except Exception as e:
        print(f"Error loading to database: {e}")

def load_runner_plays_to_db(df):
    # Ensure column names are all lowercase to match our SQL definition
    df.columns = [c.lower() for c in df.columns]
    
    try:
        df.to_sql('runner_play', engine, if_exists='append', index=False)
        print("Successfully loaded runner plays into the database.")
    except Exception as e:
        print(f"Error loading to database: {e}")


In [28]:
#df_games.drop('source_folder_date', axis=1, inplace=True)
# Run the loader with your deduplicated DataFrame
load_games_to_db(df_games)

Successfully loaded games into the database.


In [30]:
df_linescores_raw[['inning', 'half']].value_counts()

inning  half
1       0       2952
4       0       2952
6       1       2952
        0       2952
5       0       2952
4       1       2952
5       1       2952
3       1       2952
        0       2952
2       1       2952
        0       2952
1       1       2952
7       0       2949
        1       2949
8       0       2945
        1       2945
9       0       2944
        1       2944
10      1        216
        0        216
11      0         80
        1         80
12      0         16
        1         16
13      0          5
        1          5
14      0          2
        1          2
15      0          2
        1          2
16      0          1
        1          1
17      0          1
        1          1
18      0          1
        1          1
Name: count, dtype: int64

In [33]:
df_linescores_raw['runs'].value_counts()

runs
0.0     37899
1.0      7712
2.0      3736
3.0      1663
4.0       782
5.0       336
6.0       164
7.0        57
8.0        34
9.0        14
10.0        7
11.0        1
13.0        1
Name: count, dtype: int64

In [34]:
df_linescores_raw['hits'].value_counts()

hits
0     24564
1     16699
2      7727
3      3073
4      1072
5       397
6       135
7        60
8        15
9         3
10        2
12        1
Name: count, dtype: int64

In [35]:
df_linescores_raw['errors'].value_counts()

errors
0    50885
1     2683
2      168
3       12
Name: count, dtype: int64

In [36]:
df_linescores_raw['leftOnBase'].value_counts()

leftOnBase
0    25340
1    18465
2     8277
3     1666
Name: count, dtype: int64

In [37]:
df_linescores_raw['battingteamid'].value_counts()

battingteamid
141     1912
136     1904
119     1869
147     1835
158     1834
116     1833
111     1807
113     1801
109     1801
114     1801
135     1789
143     1786
112     1779
140     1775
108     1772
118     1769
115     1766
144     1766
133     1764
137     1762
142     1759
117     1758
138     1757
121     1755
146     1752
134     1752
139     1744
145     1744
120     1739
110     1737
341       18
241       18
5434      18
562       18
159       10
160       10
105        9
235        9
459        9
343        7
Name: count, dtype: int64

In [38]:
df_linescores_raw.isnull().sum()

gamePk                   0
inning                   0
half                     0
battingteamid            0
runs                  1342
hits                     0
errors                   0
leftOnBase               0
source_folder_date       0
dtype: int64

In [126]:
# Removing Duplicate Games

# Check how many games are duplicated across dates
dup_counts = df_linescores_raw[['gamePk', 'inning', 'half']].value_counts()
print(f"Linescores appearing more than once: {(dup_counts > 1).sum()}")

# Remove duplicates
df_linescores = df_linescores_raw.sort_values(['gamePk', 'inning', 'half', 'source_folder_date'], ascending=[True, True, True, False])
df_linescores = df_linescores.drop_duplicates(subset=['gamePk', 'inning', 'half'], keep='first')

# Validation: should be 0
print(f"Duplicates remaining: {df_linescores[['gamePk', 'inning', 'half']].duplicated().sum()}")

Linescores appearing more than once: 74
Duplicates remaining: 0


In [ ]:
def process_linescores(linescores_df):

    df = linescores_df.copy()
    # Convert NaNs to 0
    df['runs'] = df['runs'].fillna(0).astype(int)
    # Ensure chronological order for running total
    df = df.sort_values(['gamePk', 'inning', 'half']).copy()

    # Calculate cumulative runs for batting team
    df['cumulative_runs_batting_team'] = df.groupby(['gamePk', 'battingteamid'])['runs'].cumsum()
    df['cumulative_runs_total'] = df.groupby('gamePk')['runs'].cumsum()

    df['battingteam_score'] = df.groupby(['gamePk', 'battingteamid'])['cumulative_runs_batting_team'].shift(1).fillna(0).astype(int)
    df['total_score'] = df.groupby(['gamePk'])['cumulative_runs_total'].shift(1).fillna(0).astype(int)


    df['battingteam_score_diff'] = df['battingteam_score'] - (df['total_score'] - df['battingteam_score'])
    df_final = df.drop(columns=['cumulative_runs_batting_team', 'cumulative_runs_total', 'total_score', 'source_folder_date'])
    return df_final

df_linescores = process_linescores(df_linescores)


In [123]:
load_linescores_to_db(df_linescores)

Error loading to database: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "linescore_pkey"
DETAIL:  Key (gamepk, inning, battingteamid)=(776135, 1, 117) already exists.

[SQL: INSERT INTO linescore (gamepk, inning, half, battingteamid, runs, hits, errors, leftonbase, battingteam_score, battingteam_score_diff) VALUES (%(gamepk__0)s, %(inning__0)s, %(half__0)s, %(battingteamid__0)s, %(runs__0)s, %(hits__0)s, %(errors__0)s, % ... 202690 characters truncated ... , %(errors__999)s, %(leftonbase__999)s, %(battingteam_score__999)s, %(battingteam_score_diff__999)s)]
[parameters: {'runs__0': 1, 'leftonbase__0': 3, 'battingteamid__0': 117, 'half__0': 0, 'hits__0': 2, 'battingteam_score_diff__0': 0, 'inning__0': 1, 'gamepk__0': 776135, 'battingteam_score__0': 0, 'errors__0': 0, 'runs__1': 1, 'leftonbase__1': 0, 'battingteamid__1': 108, 'half__1': 1, 'hits__1': 1, 'battingteam_score_diff__1': -1, 'inning__1': 1, 'gamepk__1': 776135, 'battingteam_score__1': 0, 'err

# Correct Column Names for Runner play table

In [107]:
# Check initial duplicates value
dup_counts = df_runners_raw[['gamePk', 'atBatIndex', 'playIndex', 'runnerid', 'start', 'end']].value_counts()
print(f"Runner plays appearing more than once: {(dup_counts > 1).sum()}")

# Remove duplicates
df_runners = df_runners_raw.sort_values(['gamePk', 'atBatIndex', 'playIndex', 'runnerid', 'start', 'end', 'source_folder_date'], ascending=[True, True, True, True, True, True, False])
df_runners = df_runners.drop_duplicates(subset=['gamePk', 'atBatIndex', 'playIndex', 'runnerid', 'start', 'end'], keep='first')

# Print duplicates remaining
dup_counts = df_runners[['gamePk', 'atBatIndex', 'playIndex', 'runnerid', 'start', 'end']].value_counts()
print(f"Duplicate runner plays remaining: {(dup_counts > 1).sum()}")

# Rename columns
df_runners = df_runners.rename(columns={
    'originBase': 'startbase',
    'end': 'endbase',
    'isOut': 'is_out',
})

# Set cols to lowercase to match database naming
df_runners.columns = [c.lower() for c in df_runners.columns]


Runner plays appearing more than once: 848
Duplicate runner plays remaining: 0


In [108]:
# This tells us we need to change null values for startbase, start, endbase, is_out. 
# outbase and outnumber can be null on instances where the runner isn't out.
# movementeason can also be null when the runner does not move.

df_runners.isnull().sum()

gamepk                       0
atbatindex                   0
playindex                    0
playid                      46
runnerid                     0
runnerfullname               0
startbase               224460
start                   222755
endbase                 156568
event                        0
eventtype                    0
movementreason          222692
is_out                      63
outbase                 145192
outnumber               145192
isscoringevent               0
rbi                          0
earned                       0
teamunearned                 0
responsiblepitcherid    274873
source_folder_date           0
dtype: int64

In [109]:
# This shows us we need to change 4B to HM for consistency
df_runners[['outbase', 'outnumber']].value_counts()

outbase  outnumber
1B       3.0          49522
         1.0          49120
         2.0          47202
2B       2.0           4263
         1.0           2804
         3.0           2085
4B       2.0            367
3B       2.0            330
         3.0            253
4B       3.0            214
3B       1.0            201
4B       1.0            144
Name: count, dtype: int64

In [110]:
# Need to fill empty base valeus with 'B'
df_runners['startbase'] = df_runners['startbase'].fillna('B')
df_runners['endbase'] = df_runners['endbase'].fillna('B')
df_runners['start'] = df_runners['start'].fillna('B')

# Standardize 4B/score to HM (Home)
df_runners['endbase'] = df_runners['endbase'].replace('score', 'HM')
df_runners['outbase'] = df_runners['outbase'].replace('4B', 'HM')

# Check value counts
print(df_runners[['startbase','start', 'endbase']].value_counts())

startbase  start  endbase
B          B      B          145432
                  1B          60117
1B         1B     2B          25788
2B         2B     3B          13998
B          B      2B           9578
1B         1B     B            9143
3B         3B     HM           8978
B          B      HM           6817
2B         2B     HM           4455
1B         2B     3B           3573
           1B     3B           3073
2B         3B     HM           2941
1B         1B     HM           2298
           3B     HM           1206
B          1B     2B           1144
2B         2B     B             818
B          B      3B            811
3B         3B     B             530
B          1B     B             285
           2B     3B            160
1B         2B     B             157
                  HM            117
           3B     B              90
2B         3B     B              66
B          1B     3B             54
           2B     B              44
           3B     HM             10
1B

In [111]:
# After looking further, these null values aren't a mistake. They are the result of a strikeout where the runner still moved bases due to a wild pitch.
null_outs = df_runners[df_runners['is_out'].isna()]

print(f"Number of rows with null is_out: {len(null_outs)}")
if not null_outs.empty:
    display(null_outs[['gamepk', 'atbatindex', 'runnerfullname', 'event', 'eventtype', 'source_folder_date']])

Number of rows with null is_out: 63


,gamepk,atbatindex,runnerfullname,event,eventtype,source_folder_date
66266,776215,35,Otto Kemp,Strikeout,strikeout,2025-09-23
66205,776224,53,Will Benson,Strikeout,strikeout,2025-09-23
261564,776226,30,Josh Jung,Strikeout,strikeout,2025-09-21
261958,776228,28,Isaac Paredes,Strikeout,strikeout,2025-09-21
247085,776294,8,Geraldo Perdomo,Strikeout,strikeout,2025-09-16
...,...,...,...,...,...,...
4637,779027,65,Henry Davis,Strikeout,strikeout,2025-03-15
6097,779091,67,Miguel Andujar,Strikeout,strikeout,2025-03-15
166720,779119,34,Adley Rutschman,Strikeout,strikeout,2025-03-18
13447,787927,45,Brendan Rodgers,Strikeout,strikeout,2025-03-24


In [112]:
df_runners.isnull().sum()

gamepk                       0
atbatindex                   0
playindex                    0
playid                      46
runnerid                     0
runnerfullname               0
startbase                    0
start                        0
endbase                      0
event                        0
eventtype                    0
movementreason          222692
is_out                      63
outbase                 145192
outnumber               145192
isscoringevent               0
rbi                          0
earned                       0
teamunearned                 0
responsiblepitcherid    274873
source_folder_date           0
dtype: int64

In [113]:
df_runners[['event', 'eventtype']].value_counts()

event                         eventtype                   
Single                        single                          56260
Strikeout                     strikeout                       49724
Groundout                     field_out                       42358
Flyout                        field_out                       29249
Walk                          walk                            25977
Double                        double                          17563
Lineout                       field_out                       12756
Home Run                      home_run                        10740
Pop Out                       field_out                       10232
Forceout                      force_out                        9416
Grounded Into DP              grounded_into_double_play        8076
Stolen Base 2B                stolen_base_2b                   3793
Sac Fly                       sac_fly                          3645
Hit By Pitch                  hit_by_pitch               

In [114]:
df_runners[['movementreason']].value_counts()

movementreason                
r_adv_force                       36132
r_adv_play                        26322
r_force_out                        7936
r_stolen_base_2b                   3768
r_caught_stealing_2b                899
r_adv_throw                         558
r_stolen_base_3b                    523
r_thrown_out                        487
r_runner_out                        466
r_doubled_off                       348
r_defensive_indiff                  331
r_out_stretching                    237
r_pickoff_1b                        209
r_pickoff_caught_stealing_2b        178
r_pickoff_error_1b                  137
r_caught_stealing_3b                117
r_rundown                            78
r_pickoff_2b                         51
r_pickoff_error_2b                   50
r_pickoff_caught_stealing_3b         34
r_out_returning                      27
r_pickoff_3b                         26
r_caught_stealing_home               25
r_stolen_base_home                   21
r_hbr    

In [115]:
# Aggregate data into one summarizing runner play per runner per at bat

# Ensure data is sorted chronologically by playIndex so 'first' and 'last' are accurate
df_runners = df_runners.sort_values(['gamepk', 'atbatindex', 'playindex'])

# Helper for calculating reached base
def calculate_reached_base(group):
    # Find all segments in this play where the runner was SAFE
    safe_segments = group[group['is_out'] == False]
    
    if not safe_segments.empty:
        # Return the 'endbase' of the very last safe movement
        return safe_segments['endbase'].iloc[-1]
    else:
        # If they were never safe on this play (immediately out),
        # their reachedbase is just the base they started on.
        return group['startbase'].iloc[0]
    
def uniq_join(s):
    """
    Join distinct movement/event labels observed during the play.
    Duplicate labels are collapsed; distinct labels (e.g. stolen_base_2b vs _3b)
    are preserved.
    """
    vals = sorted(set(s.dropna().astype(str)))
    return ",".join(vals) if vals else None


runner_play_final = df_runners.groupby(['gamepk', 'atbatindex', 'playindex', 'runnerid']).agg(
    # startbase: The very first base they occupied when the play began
    startbase=('startbase', 'first'),
    
    # endbase: The very last base they reached (regardless of if they were out)
    endbase=('endbase', 'last'),

    # runnerfullname
    runnerfullname=('runnerfullname', 'first'),
    
    # reachedbase: The furthest base they reached SAFELY
    # Logic: Look at all 'endbase' values where is_out was False, pick the last one.
    # If they were out immediately, default to their startbase.
    reachedbase=('endbase', lambda x: calculate_reached_base(df_runners.loc[x.index])),
    
    # Decided to use the event that started the play and movement reason from the last movement the runner made
    eventtype=('eventtype', uniq_join),
    movementreason=('movementreason', uniq_join),
    
    # is_out: If they were out at ANY point during their movement, the play is an out for them
    is_out=('is_out', 'max'),
    
    # playid: Should be identical unless null
    playid=('playid', 'first')
).reset_index()

In [116]:
# Add in calculated metrics

# 1. is_risp (Runners in Scoring Position)
runner_play_final['is_risp'] = runner_play_final['startbase'].isin(['2B', '3B'])

# 2. is_firsttothird
runner_play_final['is_firsttothird'] = (
    (runner_play_final['startbase'] == '1B') & 
    (runner_play_final['reachedbase']=='3B') & 
    (~runner_play_final['eventtype'].str.contains('home_run', na=False)) &
    (runner_play_final['is_out'] == False)
)

# 3. is_secondtohome
runner_play_final['is_secondtohome'] = (
    (runner_play_final['startbase'] == '2B') & 
    (runner_play_final['reachedbase'] == 'HM') & 
    (~runner_play_final['eventtype'].str.contains('home_run', na=False)) &
    (runner_play_final['is_out'] == False)
)

In [124]:
load_runner_plays_to_db(runner_play_final)


Successfully loaded runner plays into the database.


In [118]:
jose_stolen_bases = df_runners_raw[
    df_runners_raw['movementReason'].str.contains('stolen_base', na=False)
    & (df_runners_raw['runnerfullName'] == 'José Caballero')
]
jose_stolen_bases['movementReason'].value_counts()



movementReason
r_stolen_base_2b    40
r_stolen_base_3b    14
Name: count, dtype: int64

In [119]:
dup_counts = jose_stolen_bases[['gamePk', 'atBatIndex', 'playIndex', 'runnerid', 'start', 'end']].value_counts()
print(f"Runner plays appearing more than once: {(dup_counts > 1).sum()}")

Runner plays appearing more than once: 0


In [120]:
jose_stolen_bases[['event', 'eventType', 'movementReason']].value_counts(dropna=False)




event           eventType       movementReason  
Stolen Base 2B  stolen_base_2b  r_stolen_base_2b    40
Stolen Base 3B  stolen_base_3b  r_stolen_base_3b    14
Name: count, dtype: int64

In [104]:
jose_stolen_bases.head()

,gamePk,atBatIndex,playIndex,playId,runnerid,runnerfullName,originBase,start,end,event,...,movementReason,isOut,outBase,outNumber,isScoringEvent,rbi,earned,teamUnearned,responsiblepitcherid,source_folder_date
5096,778968,53,1,07789686-0546-0013-000c-f08cd117d70a,676609,José Caballero,1B,1B,2B,Stolen Base 2B,...,r_stolen_base_2b,False,NaN,NaN,False,False,False,False,NaN,2025-03-15
19689,776925,84,6,892f60e5-64f4-31dd-aae9-7e578cfb8cb2,676609,José Caballero,1B,1B,2B,Stolen Base 2B,...,r_stolen_base_2b,False,NaN,NaN,False,False,False,False,NaN,2025-08-01
27624,778967,45,4,07789676-0466-0023-000c-f08cd117d70a,676609,José Caballero,1B,1B,2B,Stolen Base 2B,...,r_stolen_base_2b,False,NaN,NaN,False,False,False,False,NaN,2025-03-13
31845,777030,53,2,a8005f0e-c08c-3a13-bd61-469331e86cd6,676609,José Caballero,1B,1B,2B,Stolen Base 2B,...,r_stolen_base_2b,False,NaN,NaN,False,False,False,False,NaN,2025-07-23
43254,778074,11,1,09430325-01b5-371b-9bc2-b48d96282759,676609,José Caballero,2B,2B,3B,Stolen Base 3B,...,r_stolen_base_3b,False,NaN,NaN,False,False,False,False,NaN,2025-05-03


In [121]:
jose_stolen_bases = jose_stolen_bases.rename(columns={'gamePk': 'gamepk'})
jose = jose_stolen_bases.merge(df_games[['gamepk', 'gametype']], on='gamepk', how='left')
jose['gametype'].value_counts(dropna=False)
jose_R = jose[jose['gametype']=='R']
jose_R['movementReason'].value_counts()
len(jose_R)



49

In [106]:
grp = (df_runners_raw[df_runners_raw['runnerid'] == 676609]
       .groupby(['gamePk','atBatIndex','playIndex','runnerid'])['movementReason']
       .agg(lambda s: set(s.dropna()))
)

miss_candidates = grp[grp.apply(lambda x: any('stolen_base' in r for r in x) and len(x) > 1)]
miss_candidates.head(20)
len(miss_candidates)


3